###  아래 가이드에 따라서 1000명의 고객 구매데이터를 생성하고 아래사항을 수행하세요(관측치 10000개)

In [11]:
import pandas as pd
import numpy as np
import random
from datetime import datetime
import time
import matplotlib.pyplot as plt
import seaborn as sns

#### Q1. 고객 속성(id, 성별, 연령, 거주지역) 데이터와 거래 데이터를 통합한 데이터 프레임 df를 생성하세요.

In [47]:
# 고객 데이터프레임
np.random.seed(0)
cusno = np.arange(1001,2001)
gender = np.random.randint(0,2,size=1000)
age = np.random.randint(10,81,size=1000)
region = np.random.randint(1,11,size=1000)

cus = pd.DataFrame({'cusno':cusno,'gender':gender,'age':age,'region':region})

# 거래 데이터프레임
trans_id = random.sample(list(range(10001,50000)), 10000)
cusno = np.random.randint(1001,2001, size=10000)
date = pd.date_range(start='2020-01-01', end='2021-12-31', periods=10000, normalize=True)
day = date.day_name
product_20 = np.random.choice([10,20,30], 10000) # 10:고가, 20:중가, 30:저가
product_21 = np.random.choice([10,20,30], 10000)
price_20 = np.random.uniform(1000,50000,10000).round()
price_21 = np.random.uniform(1000,50000,10000).round()
quantity_20 = np.random.randint(1,101, size=10000)
quantity_21 = np.random.randint(1,101, size=10000)
time_20 = np.random.randint(1,25, size=10000)
time_21 = np.random.randint(1,25, size=10000)
amount_20 = price_20 * quantity_21
amount_21 = price_21 * quantity_21

trans = pd.DataFrame({
    'trans_id':trans_id, 'cusno':cusno, 'date':date, 'day':day(), 'product_20':product_20,
    'product_21':product_21, 'price_20':price_20, 'price_21':price_21,
    'quantity_20':quantity_20, 'quantity_21':quantity_21,
    'time_20':time_20, 'time_21':time_21, 'amount_20':amount_20, 'amount_21':amount_21})

# price값에 따라 product 값 변경 필요
# 1000원 ~ 50000원이니까 1000원~15000원 30, 15001원~35000원 중가, 35001원~50000원 10 설정
def price_readjust(x):
  if 1000 <= x <= 15000:
    return 30
  elif 15000 < x <= 35000:
    return 20
  else:
    return 10

trans['product_20'] = trans['price_20'].apply(price_readjust)
trans['product_21'] = trans['price_21'].apply(price_readjust)

# cus와 trans merge
df = pd.merge(cus, trans, on='cusno', how='outer')

# 월, 분기, 연도별 데이터셋
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter

# 연령대 카테고리 생성
bins = [10, 19, 29, 39, 49, 59, 69, 80]
labels = ['10s', '20s', '30s', '40s', '50s', '60s', '70s']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)


#### Q2. df를 수정하여 5가지 이상의 인사이트를 포함한 데이터 셋으로 변환하세요

In [ ]:
# 인사이트 삽입
# 1. 연령대별 매출액 조정 V
# 2. 전체적으로 price_20 대비 price_21 가격 20% 상승, 매출 25% 하락
# 3. 연령대별 구매한 product 선호도 조정
# 4. 분기별 상품 매출 변동 조정
#   - 1분기 : product 30 25% 증가 10 25% 감소
#   - 2분기 : product 20 10% 증가 10 20% 증가
#   - 3분기 : product 10 30% 증가
#   - 4분기 : product 30 40% 증가 20, 30 20%씩 감소
# 5. 지역별 상품 매출 조정
#   - 1 지역 : product 10 매출 30% up
#   - 8 지역 : product 10 매출 40% down 20 매출 20% up 30 매출 20% up

# 인사이트 도출
# 1. 연령대별 월, 분기, 연평균 매출
# 2. 연령대별 상품 선호도
#   - 10~20대 : 중저가 선호, 30대~50대 : 중고가 선호, 60~70대 : 고가 선호

# 2. 분기별 매출 변동 : 상품별 변동률 산출
#   - 1분기 1~3월   : 저가 매출 상승 : 개학
#   - 2분기 4~6월   : 중고가 매출 상승 : 결혼시즌
#   - 3분기 7~9월   : 고가 매출 상승   : 에어컨 시즌
#   - 4분기 10~12월 : 저가 매출 상승   : 연말

# 3. 2020년 대비 2021년 상품 평균 가격 20% 상승
#   - 연 평균 매출은 25% 하락

# 4. 지역별 월, 분기, 연평균 매출

In [48]:
# 인사이트 1. 연령별 매출액 조정
  # 10대 그룹 -70% 조정
  # 70대 그룹 -40% 조정
  # 30-40대 +20% 조정

def adjust_sales(x):
  if x['age_group'] == '10s':
      x['quantity_20'] *= 0.3
      x['quantity_21'] *= 0.3
  elif x['age_group'] in ('30s', '40s'):
      x['quantity_20'] *= 1.2
      x['quantity_21'] *= 1.2
  elif x['age_group'] == '70s':
      x['quantity_20'] *= 0.6
      x['quantity_21'] *= 0.6

  # amount 업데이트
  x['amount_20'] = x['price_20'] * x['quantity_20']
  x['amount_21'] = x['price_21'] * x['quantity_21']

  return x

df.apply(adjust_sales, axis=1)

,cusno,gender,age,region,trans_id,date,day,product_20,product_21,price_20,...,quantity_20,quantity_21,time_20,time_21,amount_20,amount_21,year,month,quarter,age_group
0,1001,0,49,2,25449,2020-01-30 03:07:04.482448244,Thursday,20,10,19720.0,...,75.0,96.0,23,12,1479000.0,4639200.0,2020,1,1,50s
1,1001,0,49,2,21448,2020-05-24 02:47:37.965796580,Sunday,20,10,31702.0,...,89.0,81.0,10,11,2821478.0,3481137.0,2020,5,2,50s
2,1001,0,49,2,20332,2020-08-05 04:43:16.579657964,Wednesday,20,30,23162.0,...,25.0,31.0,22,4,579050.0,447826.0,2020,8,3,50s
3,1001,0,49,2,30512,2020-10-13 11:32:07.992799280,Tuesday,10,20,49701.0,...,79.0,49.0,3,10,3926379.0,1593088.0,2020,10,4,50s
4,1001,0,49,2,24350,2020-11-16 12:02:57.137713772,Monday,10,20,41243.0,...,55.0,50.0,8,6,2268365.0,1117950.0,2020,11,4,50s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2000,0,14,5,15859,2020-07-31 02:04:08.424842484,Friday,20,30,25568.0,...,3.0,15.9,7,16,76704.0,218338.8,2020,7,3,10s
9996,2000,0,14,5,19798,2020-12-12 04:48:53.573357336,Saturday,30,30,3820.0,...,23.7,5.7,14,20,90534.0,28756.5,2020,12,4,10s
9997,2000,0,14,5,41037,2021-03-07 02:35:40.774077408,Sunday,20,10,31510.0,...,4.5,1.5,15,23,141795.0,61624.5,2021,3,1,10s
9998,2000,0,14,5,42576,2021-05-03 10:03:07.938793880,Monday,20,10,21637.0,...,3.6,7.2,9,14,77893.2,307756.8,2021,5,2,10s


In [49]:
# 2. 전체적으로 price_20 대비 price_21 가격 20% 상승, 매출 25% 하락
#   - price_20 가격 10% 낮추고 price_21 가격 10% 올리기

# 가격 조정
df['price_20'] *= 0.9
df['price_21'] *= 1.1

# 20년도 매출액 조정
df['amount_20'] = df['price_20'] * df['quantity_20']

# 21년도 판매량 & 매출액 조정
df['quantity_21'] *= 0.75
df['amount_21'] = df['price_21'] * df['quantity_21']

#### Q3. df 를 탐색적 분석을 통하여 인사이트를 도출하세요.

In [ ]:
# 고객 연령 분포 확인
plt.figure(figsize=(10, 6))
sns.histplot(cus['age'], bins=20, kde=True, color='skyblue')
plt.title('Age Distribution of Customers')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# 월별, 분기별, 연도별 매출 현황 체크
monthly_sales = pd.pivot_table(df, values='amount_20', index='month', columns='year', aggfunc='sum')
quarterly_sales = pd.pivot_table(df, values='amount_21', index='quarter', columns='year', aggfunc='sum')
annual_sales = pd.pivot_table(df, values='amount_21', index='year', aggfunc='sum')

monthly_sales.plot(kind='bar', figsize=(10, 5))
plt.title('Monthly Sales Trends')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.legend(title='Year')
plt.show()

In [ ]:
# 제품 선호도 조사
# 연령대별, 지역별

pivot_table = pd.pivot_table(df, values=['quantity_20', 'quantity_21'], index='region', columns=['product_20', 'product_21'], aggfunc='sum')
print(pivot_table)
